## Query execution time filtering

This sheet removes some outliers from the query execution time data. For each database (db), record volume (record_volume) and query type (query), we remove the iteration with the minimum and maximum execution time (time_in_seconds). We also filter out Cassandra's omitted Query 3.1 from 4k+ experiments. We also remove -1 as it is used to indicate that query timed out or an exception occurred.

In [263]:
import pandas as pd
import numpy as np

df = pd.read_csv('results.csv')

df["old_time_in_seconds"] = df["time_in_seconds"].copy()

df

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
0,sqlite,1000,1-1,0,0.001000,0.001000
1,sqlite,1000,1-1,1,0.001000,0.001000
2,sqlite,1000,1-1,2,0.001000,0.001000
3,sqlite,1000,1-1,3,0.001000,0.001000
4,sqlite,1000,1-1,4,0.001000,0.001000
...,...,...,...,...,...,...
8730,cassandra,512000,9-2,16,0.003040,0.003040
8731,cassandra,512000,9-2,17,0.002845,0.002845
8732,cassandra,512000,9-2,18,0.003484,0.003484
8733,cassandra,512000,9-2,19,0.002276,0.002276


In [264]:
dnf = df[df["time_in_seconds"] < 0]
dnf

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
960,sqlite,256000,3-1,1,-1.0,-1.0
981,sqlite,256000,3-3,1,-1.0,-1.0
1002,sqlite,256000,4-1,1,-1.0,-1.0
1003,sqlite,256000,4-2,1,-1.0,-1.0
1284,sqlite,512000,3-1,1,-1.0,-1.0
1305,sqlite,512000,3-3,1,-1.0,-1.0
1326,sqlite,512000,4-1,1,-1.0,-1.0
1327,sqlite,512000,4-2,1,-1.0,-1.0
2408,mysql,256000,3-1,1,-1.0,-1.0
2469,mysql,256000,4-1,1,-1.0,-1.0


In [ ]:
df.drop(df[df["db"].isin(["mysql", "sqlite"]) & (df["query"] == "4-2")].index, inplace=True)

In [265]:
# df.drop(df[df["time_in_seconds"] < 0].index, inplace=True)
dropped_duplicate_dnf = df[df["time_in_seconds"] < 0].drop_duplicates(subset=["db", "record_volume", "query", "time_in_seconds"])
dropped_duplicate_dnf = dropped_duplicate_dnf.replace({"time_in_seconds": -1}, "DNF")

df = pd.concat([df.drop(dnf.index), dropped_duplicate_dnf]).sort_index()
df

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
0,sqlite,1000,1-1,0,0.001,0.001000
1,sqlite,1000,1-1,1,0.001,0.001000
2,sqlite,1000,1-1,2,0.001,0.001000
3,sqlite,1000,1-1,3,0.001,0.001000
4,sqlite,1000,1-1,4,0.001,0.001000
...,...,...,...,...,...,...
8730,cassandra,512000,9-2,16,0.00304,0.003040
8731,cassandra,512000,9-2,17,0.002845,0.002845
8732,cassandra,512000,9-2,18,0.003484,0.003484
8733,cassandra,512000,9-2,19,0.002276,0.002276


In [266]:
df[df["time_in_seconds"] == "DNF"]

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
960,sqlite,256000,3-1,1,DNF,-1.0
981,sqlite,256000,3-3,1,DNF,-1.0
1002,sqlite,256000,4-1,1,DNF,-1.0
1003,sqlite,256000,4-2,1,DNF,-1.0
1284,sqlite,512000,3-1,1,DNF,-1.0
1305,sqlite,512000,3-3,1,DNF,-1.0
1326,sqlite,512000,4-1,1,DNF,-1.0
1327,sqlite,512000,4-2,1,DNF,-1.0
2408,mysql,256000,3-1,1,DNF,-1.0
2469,mysql,256000,4-1,1,DNF,-1.0


In [267]:
df.drop(df[(df["db"] == "cassandra") & (df["record_volume"] != 1000) & (df["query"] == "3-1")].index, inplace=True)

In [269]:
gb = df.groupby(["db", "record_volume", "query"])["time_in_seconds"]
gb.describe()

count unique       top freq
db       record_volume query                            
arangodb 1000          1-1    20.0   20.0  0.001766  1.0
                       1-2    20.0   20.0  0.001571  1.0
                       1-3    20.0   20.0   0.00049  1.0
                       1-4    20.0   20.0  0.005032  1.0
                       10     20.0   20.0  0.030094  1.0
...                            ...    ...       ...  ...
sqlite   512000        6      20.0   20.0    17.452  1.0
                       7      20.0   18.0     10.31  2.0
                       8      20.0   16.0      1.07  3.0
                       9-1    20.0   19.0     1.736  2.0
                       9-2    20.0   15.0     0.992  3.0

[460 rows x 4 columns]

In [270]:
idxmin = gb.idxmin()
df.loc[idxmin]

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
4417,arangodb,1000,1-1,17,0.000414,0.000414
4433,arangodb,1000,1-2,13,0.001397,0.001397
4445,arangodb,1000,1-3,5,0.00025,0.000250
4478,arangodb,1000,1-4,18,0.00142,0.001420
4778,arangodb,1000,10,18,0.029474,0.029474
...,...,...,...,...,...,...
1353,sqlite,512000,6,6,8.063,8.063000
1369,sqlite,512000,7,2,10.112,10.112000
1390,sqlite,512000,8,3,0.83,0.830000
1419,sqlite,512000,9-1,12,1.322,1.322000


In [271]:
idxmax = gb.idxmax()
df.loc[idxmax]

,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds
4400,arangodb,1000,1-1,0,0.001766,0.001766
4435,arangodb,1000,1-2,15,0.002755,0.002755
4447,arangodb,1000,1-3,7,0.000558,0.000558
4460,arangodb,1000,1-4,0,0.005032,0.005032
4769,arangodb,1000,10,9,0.036892,0.036892
...,...,...,...,...,...,...
1348,sqlite,512000,6,1,17.452,17.452000
1368,sqlite,512000,7,1,13.287,13.287000
1397,sqlite,512000,8,10,1.101,1.101000
1415,sqlite,512000,9-1,8,1.844,1.844000


In [273]:
df["time_in_seconds"] = df["time_in_seconds"].loc[(~df.index.isin(idxmin) & ~df.index.isin(idxmax)) | (df["time_in_seconds"] == "DNF")]
df["extra_label"] = df["time_in_seconds"][df["time_in_seconds"] == "DNF"].copy()
df.replace({"time_in_seconds": "DNF"}, 300, inplace=True)
df

C:\Users\marti\AppData\Local\Temp\ipykernel_71184\1051173903.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"time_in_seconds": "DNF"}, 300, inplace=True)


,db,record_volume,query,iteration,time_in_seconds,old_time_in_seconds,extra_label
0,sqlite,1000,1-1,0,NaN,0.001000,NaN
1,sqlite,1000,1-1,1,0.001000,0.001000,NaN
2,sqlite,1000,1-1,2,0.001000,0.001000,NaN
3,sqlite,1000,1-1,3,0.001000,0.001000,NaN
4,sqlite,1000,1-1,4,0.001000,0.001000,NaN
...,...,...,...,...,...,...,...
8730,cassandra,512000,9-2,16,0.003040,0.003040,NaN
8731,cassandra,512000,9-2,17,0.002845,0.002845,NaN
8732,cassandra,512000,9-2,18,0.003484,0.003484,NaN
8733,cassandra,512000,9-2,19,NaN,0.002276,NaN


In [274]:
df.to_csv('results_filtered.csv', index=False)